In [1]:
from astropy.modeling import models, Fittable1DModel, Parameter
from astropy.modeling import fitting
import astropy.units as u
import astropy.constants as const

from spectacle.spectrum import SpectrumModel

import numpy as np
import matplotlib.pyplot as plt

TypeError: could not convert reader output to LineRegistry class.

In [ ]:
%matplotlib notebook
# plt.rcParams["figure.figsize"] = [8, 16]
# plt.rcParams["figure.dpi"] = 80

## Control Spectrum

This spectrum acts as the standard to which the dynamic spectrum will be compared in the correlation metrics.

In [ ]:
import astropy.units as u

# velocity = np.linspace(1210, 1220, 1001) * u.Angstrom
velocity = np.linspace(-800, 800, 201) * u.Unit("km/s")

In [ ]:
from spectacle.models import TauProfile
from astropy.modeling.models import Voigt1D
from astropy.modeling.fitting import LevMarLSQFitter

line = dict(name="HI1216")

spectrum = SpectrumModel(center=line['lambda_0']).add_line(**line)

voigt = Voigt1D(x_0=0.0001, amplitude_L=0.17, fwhm_G=40, fwhm_L=8)

fitter = LevMarLSQFitter()
print(spectrum.flux)

new_voigt = fitter(voigt, velocity, spectrum.flux(velocity))

f, ax = plt.subplots()

ax.plot(velocity, spectrum.flux(velocity))
# ax.scatter(velocity, voigt(velocity.value))

print(new_voigt)
# ax.plot(velocity, new_voigt(velocity))



# fwhm = line.fwhm(velocity, velocity=True)
# print(fwhm)

# ax.axvline(line['lambda_0']-fwhm*0.5)
# ax.axvline(line['lambda_0']+fwhm*0.5)

# ax.axvline(line['lambda_0']-fwhm)
# ax.axvline(line['lambda_0']+fwhm)

# mask = spectrum.line_mask(velocity)
# ax.scatter(velocity[mask], spectrum.tau.velocity_space(velocity, center=line['lambda_0'])[mask])

# # print(line.dv90(wavelength))